In [1]:
import openmc
import openmc.deplete as od
import math
import pandas as pd
import numpy as np
import openmc.mgxs as mgxs

In [2]:
fuel = openmc.Material(name="uo2")
fuel.add_element("U", 1, percent_type="ao", enrichment=4.25)
fuel.add_element("O", 2)
fuel.set_density("g/cc", 10.4)

clad = openmc.Material(name="clad")
clad.add_element("Zr", 1)
clad.set_density("g/cc", 6)

water = openmc.Material(name="water")
water.add_element("O", 1)
water.add_element("H", 2)
water.set_density("g/cc", 1.0)
water.add_s_alpha_beta("c_H_in_H2O")

radii = [0.42, 0.45]
fuel.volume = np.pi * radii[0] ** 2
clad.volume = np.pi * (radii[1]**2 - radii[0]**2)
water.volume = 1.24**2 - (np.pi * radii[1]**2)

materials = openmc.Materials([fuel, clad, water])

In [3]:
pin_surfaces = [openmc.ZCylinder(r=r) for r in radii]
pin_univ = openmc.model.pin(pin_surfaces, materials)
bound_box = openmc.rectangular_prism(1.24, 1.24, boundary_type="reflective")
root_cell = openmc.Cell(fill=pin_univ, region=bound_box)
geometry = openmc.Geometry([root_cell])

In [4]:
settings = openmc.Settings()
settings.particles = 1000
settings.inactive = 10
settings.batches = 50

In [23]:
groups = mgxs.EnergyGroups()
groups.group_edges = np.array([0.,20.0e6])
fission = mgxs.FissionXS(domain=fuel, groups=groups, by_nuclide=True)
capture = mgxs.CaptureXS(domain=fuel, groups=groups, by_nuclide=True)

tallies = openmc.Tallies()
tallies += fission.tallies.values()
tallies += capture.tallies.values()


In [5]:
model = openmc.Model(geometry, materials, settings)

In [10]:
model.materials[0]

Material
	ID             =	1
	Name           =	uo2
	Temperature    =	None
	Density        =	10.4 [g/cc]
	S(a,b) Tables  
	Nuclides       
	U234           =	0.0003845183882296898 [ao]
	U235           =	0.04301995213827323 [ao]
	U238           =	0.9563984774295304 [ao]
	U236           =	0.000197052043966672 [ao]
	O16            =	1.999242     [ao]
	O17            =	0.000758     [ao]

In [9]:
model.export_to_xml()

In [6]:
from openmc.deplete import MicroXS

In [7]:
#import pdb
#pdb.set_trace()
micro_xs = MicroXS.from_model(model, fuel)

# b /home/ooblack/projects/openmc/openmc/deplete/microxs.py:92

/home/ooblack/projects/openmc/openmc/mixin.py:67: IDWarning: Another Filter instance already exists with id=1.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:67: IDWarning: Another Filter instance already exists with id=2.
  warn(msg, IDWarning)


                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [9]:
test_xs = MicroXS.from_csv('test_reference.csv')

In [10]:
np.all(micro_xs == test_xs)

True

In [11]:
micro_xs['(n,gamma)'].loc['U238']

0.8652742788116056

In [12]:
test_xs['(n,gamma)'].loc['U238']

0.8652742788116056

In [8]:
micro_xs.to_csv('test_reference.csv')

In [50]:
test_xs['(n,gamma)'].loc['U238']

0.8652742788116056

In [49]:
micro_xs['(n,gamma)'].loc['U238']

0.8652742788116056

In [43]:
model.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

RuntimeError: Could not find material 4 specified on tally filter.

## Get 1G XS

In [26]:
# Load the last statepoint file
sp = openmc.StatePoint('statepoint.50.h5')


# Load the tallies from the statepoint into each MGXS object
fission.load_from_statepoint(sp)
capture.load_from_statepoint(sp)
# Close the statepoint file now that we're done getting info
sp.close()

In [33]:
fission_df

,material,group in,nuclide,mean,std. dev.
0,4,1,U234,0.495319,0.003178
1,4,1,U235,49.109559,0.298703
2,4,1,U238,0.105793,0.000924
3,4,1,U236,0.323154,0.005135
4,4,1,O16,0.000000,0.000000
5,4,1,O17,0.000000,0.000000


In [32]:
fission_df = fission.get_pandas_dataframe(xs_type='micro')
capture_df = capture.get_pandas_dataframe(xs_type='micro')

In [12]:
capture_df.index=capture_df['nuclide']
fission_df.index=fission_df['nuclide']

In [14]:
capture_df.drop(['nuclide','material','group in','std. dev.'], axis=1, inplace=True)
fission_df.drop(['nuclide','material','group in','std. dev.'], axis=1, inplace=True)

In [15]:
capture_df.rename({'mean':'(n,gamma)'}, axis=1, inplace=True)
fission_df.rename({'mean':'fission'}, axis=1, inplace=True)

In [16]:
micro_xs = pd.concat([capture_df,fission_df], axis=1)

In [6]:
micro_xs

NameError: name 'micro_xs' is not defined

In [14]:
# now try simply using the helper function
#import pdb
#pdb.set_trace()
micro_xs = openmc.deplete.MicroXS.from_model(model, reaction_domain=fuel, write_to_csv=False)

# b /home/ooblack/projects/openmc/openmc/deplete/flux_operator.py:550

AttributeError: module 'openmc.deplete' has no attribute 'MicroXS'

In [141]:
micro_xs.to_csv('micro_xs_simple.csv')

In [20]:
capture.tallies.values()

odict_values([Tally
	ID             =	8
	Name           =	
	Filters        =	MaterialFilter, EnergyFilter
	Nuclides       =	total
	Scores         =	['flux']
	Estimator      =	tracklength, Tally
	ID             =	9
	Name           =	
	Filters        =	MaterialFilter, EnergyFilter
	Nuclides       =	U234 U235 U238 U236 O16 O17
	Scores         =	['absorption']
	Estimator      =	tracklength, Tally
	ID             =	10
	Name           =	
	Filters        =	MaterialFilter, EnergyFilter
	Nuclides       =	U234 U235 U238 U236 O16 O17
	Scores         =	['fission']
	Estimator      =	tracklength])

In [ ]:
micro_xs.to

In [140]:
micro_xs.to_csv('')

'nuclide,"(n,gamma)",fission\nU234,23.518634203050645,0.49531930067650976\nU235,10.621118186344665,49.10955932965905\nU238,0.8652742788116043,0.10579281644765708\nU236,9.095623870006154,0.32315392339237936\nO16,0.0029535689693132015,0.0\nO17,0.05980775766572907,0.0\n'

In [17]:
fission.print_xs(xs_type='micro')

Multi-Group XS
	Reaction Type  =	fission
	Domain Type    =	material
	Domain ID      =	1
	Nuclide        =	U234
	Cross Sections [barns]:
            Group 1 [0.0        - 20000000.0eV]:	4.95e-01 +/- 6.42e-01%

	Nuclide        =	U235
	Cross Sections [barns]:
            Group 1 [0.0        - 20000000.0eV]:	4.91e+01 +/- 6.08e-01%

	Nuclide        =	U238
	Cross Sections [barns]:
            Group 1 [0.0        - 20000000.0eV]:	1.06e-01 +/- 8.73e-01%

	Nuclide        =	U236
	Cross Sections [barns]:
            Group 1 [0.0        - 20000000.0eV]:	3.23e-01 +/- 1.59e+00%

	Nuclide        =	O16
	Cross Sections [barns]:
            Group 1 [0.0        - 20000000.0eV]:	0.00e+00 +/- 0.00e+00%

	Nuclide        =	O17
	Cross Sections [barns]:
            Group 1 [0.0        - 20000000.0eV]:	0.00e+00 +/- 0.00e+00%





/home/ooblack/projects/openmc/openmc/tallies.py:1255: RuntimeWarning: invalid value encountered in true_divide
  data = self.std_dev[indices] / self.mean[indices]


In [136]:
capture_df

,mean
nuclide,
U234,23.518634
U235,10.621118
U238,0.865274
U236,9.095624
O16,0.002954
O17,0.059808
